### Audio preprocessing

In [1]:
import torch

from examples.cases import example_utils
from examples.cases.call_stats import DialogStats, call_statistic
from expert.data.annotation import speech_to_text
from expert.data.diarization.speaker_diarization import SpeakerDiarization

/home/izs/miniconda3/envs/relevant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/izs/.local/lib/python3.9/site-packages/whisper/timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [2]:
file_path = "examples/cases/test_aud.wav"

Diarization = SpeakerDiarization(audio=file_path, device=torch.device("cpu"))
speakers = Diarization.apply()

In [3]:
transcribation = speech_to_text.transcribe_video(file_path, lang="ru")

In [4]:
all_words = speech_to_text.get_all_words(transcribation)[0]

In [5]:
timestamps = []
for speaker in speakers:
    for elem in speakers[speaker]:
        timestamps.append((elem, speaker))

In [11]:
sentences = example_utils.sentences_with_time(timestamps, all_words)

Connecting sentences with speech intevals form diarization

In [12]:
ci = example_utils.place_words(timestamps, sentences[1])

for num, i in enumerate(ci):
    timestamps[i[1]][0][0] = min(timestamps[i[1]][0][0], sentences[1][num][1])

timestamps.sort()

sentences = example_utils.sentences_with_time(timestamps, all_words)

sentences[0]

{0: (' Здравствуйте', ([0, 3], 'SPEAKER_01')),
 1: (' Мне нужна помощь. Я забыла, но вдруг внезапно обнаружила, что у меня есть карточка вашего банка. И на ней написано И тут я вспоминаю, что мне в паре карточку впалили. Думаю, ну ладно, пусть будет. И даже на ней моё имя выбито. Но когда я её получала? Что это за карточка? Актуально, я так поняла, до года, прямо на вообще. Моя фамилия',
  ([2, 36], 'SPEAKER_00')),
 2: (' Проверьте', ([36, 37], 'SPEAKER_01')),
 3: (' и скажите, что мне надо делать с этой карточкой, какие плюшки она мне дает.',
  ([37, 42], 'SPEAKER_00')),
 4: (' ДС?', ([42, 44], 'SPEAKER_01')),
 5: (' Нет, через О В секунду', ([44, 50], 'SPEAKER_00')),
 6: (' Я даже не знаю, какой кот у этой карточки. Сейчас посмотрим, скажу вам.',
  ([50, 57], 'SPEAKER_01')),
 7: ('', ([51, 55], 'SPEAKER_00')),
 8: (' Обычно на карточках расписываются. У меня тут подлинные стахи.',
  ([63, 68], 'SPEAKER_00')),
 9: ('', ([68, 69], 'SPEAKER_01')),
 10: (' Так, дата рождения ваша?', ([74

### Extracting of specific statistics

In [13]:
from examples.cases.call_stats import DialogStats, call_statistic

In [14]:
report = call_statistic(Diarization.audio, Diarization.sr, sentences[0], speakers)

Перебивал SPEAKER_00
00:02 - 00:03
Перебивал SPEAKER_01
00:36 - 00:36
Перебивал SPEAKER_00
00:37 - 00:37
Перебивал SPEAKER_01
00:42 - 00:42
Перебивал SPEAKER_00
00:44 - 00:44
Перебивал SPEAKER_01
00:50 - 00:50
Перебивал SPEAKER_00
00:51 - 00:55
Перебивал SPEAKER_01
01:08 - 01:08
Перебивал SPEAKER_00
01:16 - 01:16
Перебивал SPEAKER_00
01:32 - 01:32
Перебивал SPEAKER_01
01:35 - 01:35
Перебивал SPEAKER_00
01:47 - 01:51
Перебивал SPEAKER_00
02:07 - 02:09
Перебивал SPEAKER_00
02:12 - 02:15
Перебивал SPEAKER_01
02:25 - 02:26
Перебивал SPEAKER_01
02:27 - 02:34
Перебивал SPEAKER_00
02:43 - 02:44
Перебивал SPEAKER_00
02:45 - 02:46
Перебивал SPEAKER_01
02:47 - 02:47
Перебивал SPEAKER_00
03:03 - 03:05
Перебивал SPEAKER_01
03:11 - 03:11
Перебивал SPEAKER_00
03:13 - 03:14
Перебивал SPEAKER_01
03:22 - 03:22
Перебивал SPEAKER_01
03:55 - 03:55
Перебивал SPEAKER_00
04:17 - 04:19
Перебивал SPEAKER_01
04:28 - 04:30
Перебивал SPEAKER_01
05:06 - 05:11
Перебивал SPEAKER_01
05:23 - 05:25
Перебивал SPEAKER_00

In [15]:
import pandas as pd

In [16]:
pd.DataFrame(report)

,Продолжительность разговора сек,Тишина в начале сек,Тишина в конце сек,Длителньость тишины сек,Доля тишины,Продолжительность речи SPEAKER_00,Скорость речи SPEAKER_00,Количество интервалов SPEAKER_00,Доля речи SPEAKER_00,Продолжительность речи SPEAKER_01,Скорость речи SPEAKER_01,Количество интервалов SPEAKER_01,Доля речи SPEAKER_01,Продолжительность одновременной речи,Доля одновременной речи,Количество перебиваний SPEAKER_00,Количество перебиваний SPEAKER_01
0,414.84,0,-0.16,69.0,6.012174,218.52,128.983018,21,0.631853,227.32,127.887393,24,0.657298,50,0.120528,19,18


### Analysis of emotions by speech

In [17]:
from expert.core.congruence import audio_emotions

In [48]:
speakers = example_utils.get_rounded_intervals(speakers)

In [52]:
CoAudio = audio_emotions.audio_analysis.AudioAnalysis(video_path=file_path,
                                   stamps=speakers,
                                   speaker="SPEAKER_00",
                                   sr=44100)

Emotions of the first speaker

In [53]:
audio_0_emotions = CoAudio.predict()

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Emotions of the second speaker

In [55]:
CoAudio = audio_emotions.audio_analysis.AudioAnalysis(video_path=file_path,
                                   stamps=speakers,
                                   speaker="SPEAKER_01",
                                   sr=44100)
audio_1_emotions = CoAudio.predict()

In [56]:
emo_audio_0 = pd.DataFrame(audio_0_emotions)
emo_audio_1 = pd.DataFrame(audio_1_emotions)

In [58]:
anger_speaker_1 = emo_audio_1.loc[(emo_audio_1['audio_anger'] > emo_audio_1['audio_neutral']) & (emo_audio_1['audio_anger'] >  emo_audio_1['audio_happiness'])]
happy_speaker_1 = emo_audio_1.loc[(emo_audio_1['audio_happiness'] > emo_audio_1['audio_neutral']) & (emo_audio_1['audio_happiness'] >  emo_audio_1['audio_anger'])]

anger_speaker_0 = emo_audio_0.loc[(emo_audio_0['audio_anger'] > emo_audio_0['audio_neutral']) & (emo_audio_0['audio_anger'] >  emo_audio_0['audio_happiness'])]
happy_speaker_0 = emo_audio_0.loc[(emo_audio_0['audio_happiness'] > emo_audio_0['audio_neutral']) & (emo_audio_0['audio_happiness'] >  emo_audio_0['audio_anger'])]

In [69]:
from termcolor import colored

from datetime import datetime

Show sentences of the first speaker said with aggression

In [70]:
for time in anger_speaker_0['time_sec']:
    for elem in sentences[0]:
        if sentences[0][elem][1][1] == 'SPEAKER_00' and sentences[0][elem][1][0][0] <= time and  sentences[0][elem][1][0][1] >= time:
            print(datetime.fromtimestamp(time).strftime("%M:%S"), '  ', colored(sentences[0][elem][0], 'red'))

03:03     Сейчас по комиссии.


Show sentences of the first speaker said with happiness

In [72]:
for time in happy_speaker_0['time_sec']:
    for elem in sentences[0]:
        if sentences[0][elem][1][1] == 'SPEAKER_00' and sentences[0][elem][1][0][0] <= time and  sentences[0][elem][1][0][1] >= time:
            print(datetime.fromtimestamp(time).strftime("%M:%S"), '  ', colored(sentences[0][elem][0], 'green'))

01:32     услуг нового.
02:32     скачать это приложение. И, конечно, это не так. Я живу. Какие у меня еще варианты есть? В отделении банка. В отделении банка спрашивают, вам помогут активировать вам все. Скажите, в любой отделении или? Ну, думаю, во всех отделениях будет именно планшет, чтобы вам с их устройства помогут активировать.
02:43    
03:08     Я не очень поняла, когда
03:13     не очень поняла, когда просто пустая карта смысл в ее. Но вроде как, какая-то сумма должна быть или что-то там должно покупаться с этой карты.
03:53     За расчет
04:17     Ради того, чтобы платить не 30, а 15 рублей, нужно в месяц тратить 500 рублей. Такая замануха, ну, прямо смешно даже иногда. По логике, да? То есть, я соглашаю. Ну, и то меня я пришла выяснить один вопрос. А один. Давайте, давайте. Я не знаю, что это такое. Но, кстати, я не знаю, что это такое. Но, кстати, я не знаю, что это такое. Но, кстати, я не знаю, что это такое. Но, кстати, я не знаю, что это такое. Но, кстати, я не знаю, чт

### An example of the result obtained with AutoTM analysis for every speaker

In [ ]:
result = pd.read_csv('./AutoTM/src/result.csv')

In [76]:
result[['speakers', 'topic_1', 'topic_2']]

,speakers,topic_1,topic_2
0,speaker_00,Тарифный план по обслуживанию карты,Бонусы при использовании карты
1,speaker_01,Тарифный план по обслуживанию карты,Бонусы при использовании карты
